In [2]:
import pandas as pd
import numpy as np
import os
import re
from utils import *

import sounddevice as sd
import soundfile as sf
from scipy.io.wavfile import write

import keras
from keras.utils import to_categorical
from keras.optimizers import Adam

import tensorflow as tf

import transformers
from transformers import DistilBertTokenizer, TFDistilBertModel



tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

MAX_SEQUENCE_LENGTH = 66
BATCH_SIZE = 32
CATEGORIES = "sadness,joy,anger,fear,surprise,neutral".split(',')


def tokenize(sentences, max_length=MAX_SEQUENCE_LENGTH, padding='max_length'):

    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf"
    )

def decode(tokens):
    return tokenizer.decode(tokens)

def to_tensor(data, label = []):
    return tf.data.Dataset.from_tensor_slices((
    dict(tokenize(data)),  # Convert BatchEncoding instance to dictionary
    label
    )).batch(BATCH_SIZE).prefetch(1)

def predict(str,model):
    str = to_tensor(str)
    model.predict(str)

def predice(array,model):

    array = tf.data.Dataset.from_tensor_slices((
    dict(tokenize(array)),
    )).batch(BATCH_SIZE).prefetch(1)
    df = pd.DataFrame(columns=CATEGORIES,data=model.predict(array))
    #reorder the columns in the following order: surprise, anger, fear, sadness, joy, neutral
    df = df[['surprise','anger','fear','sadness','joy','neutral']]
    return df, df.values



NLP_model =  tf.keras.models.load_model('C:\\Users\\mario\\PROYECTO_IA\\IA-project\\Notebooks\\MARIO\\SentimentAnalysis.hdf5', custom_objects={'TFDistilBertModel': TFDistilBertModel})
# audio_model = keras.models.load_model('C:\\Users\\mario\\PROYECTO_IA\\IA-project\\Notebooks\\MARIO\\model.h5')

In [6]:


fs = 44100  # Sample rate
seconds = 8  # Duration of recording

myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
print("Recording Audio")
sd.wait()  # Wait until recording is finished
print("Finished recording")
write('output.wav', fs, myrecording)  # Save as WAV file 

!whisper --task translate --language es --model base --verbose False output.wav

with open('output.wav.txt', 'r') as file:
    data = file.read().replace('\n', '')

print(data)
df, inputs = predice(data,NLP_model)

inputs = inputs[0]

generate_sound(get_notes(inputs))
print(df)

filename = 'PRUEBAS2.wav'
# Extract data and sampling rate from file
data, fs = sf.read(filename, dtype='float32')  
sd.play(data, fs)
status = sd.wait()  # Wait until file is done playing

Recording Audio
Finished recording


C:\Users\mario\anaconda3\envs\CAPSTONE_PROJECT\lib\site-packages\whisper\transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

100%|██████████| 800/800 [00:09<00:00, 85.35frames/s]


I didn't expect this from you, you destroyed me, you had to spend the workfor the rest of the day.
1/1 [==============================] - 0s 322ms/step
   surprise    anger      fear  sadness       joy   neutral
0  0.044827  0.43381  0.050005  0.33972  0.006234  0.125404


In [3]:
#Read the data of audio_test.wav.txt file

